In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm
import scipy.stats as stats
import sys

sys.path.append("../")
import vuong_tests

In [2]:
#generate the sample
def gen_data(beta=3):
    nobs = 1000
    x = np.random.uniform(low=-3., high=3., size=(nobs,3))
    e = np.random.normal(loc=0.0, scale=1.0, size=nobs)
    y = 1*(1 + beta * x[:,1] + e >= 0) 
    return y,x,nobs

yn,xn,nobs = gen_data()

In [3]:
def compute_llr(yn,xn):
    x1n,x2n = xn[:,0:2],xn[:,1:3]
    model1 = sm.Probit(yn,sm.add_constant(x1n))
    model1_fit = model1.fit(disp=False)
    ll1 = model1.loglikeobs(model1_fit.params)

    model2 = sm.Probit(yn,sm.add_constant(x2n))
    model2_fit = model2.fit(disp=False)
    ll2 = model2.loglikeobs(model2_fit.params)

    llr = ll1.sum() - ll2.sum()
    omega2 = (ll1- ll2).var()
    return llr,np.sqrt(omega2)

yn,xn,nobs = gen_data()
print(compute_llr(yn,xn))

(0.9793026000905911, 0.048821140130840726)


In [ ]:
yn,xn,nobs = gen_data()
print(vuong_tests.bootstrap_test(yn,xn,nobs,compute_llr,hist=True))
print(vuong_tests.regular_test(yn,xn,nobs,compute_llr,hist=True))

plt.title("Comparison with bootstrap")
plt.xlabel("Test Statistic")
plt.ylabel("Density")
plt.legend()
plt.savefig('../figs/bootstrap_compare10')
plt.show()

In [8]:
reg,boot,llr, omega = vuong_tests.monte_carlo(1000,gen_data,compute_llr)
print("reg: %s, boot: %s, llr:%s, omega:%s"%(reg,boot,llr, omega))

reg: [1. 0. 0.], boot: [0.92 0.04 0.04], llr:0.04682784773699794, omega:0.03895234920325001


In [6]:
def setup_shi(yn,xn):
    x1n,x2n = xn[:,0:2],xn[:,1:3]
    model1 = sm.Probit(yn,sm.add_constant(x1n))
    model1_fit = model1.fit(disp=False)
    #print(model1_fit.params)
    ll1 = model1.loglikeobs(model1_fit.params)
    grad1 =  model1.score_obs(model1_fit.params)
    hess1 =  model1.hessian(model1_fit.params)
    
    model2 = sm.Probit(yn,sm.add_constant(x2n))
    model2_fit = model2.fit(disp=False)
    #print(model2_fit.params)
    ll2 = model2.loglikeobs(model2_fit.params)
    grad2 =  model2.score_obs(model2_fit.params)
    hess2 =  model2.hessian(model2_fit.params)
    
    k1 = len(model1_fit.params)
    k2 = len(model2_fit.params)
    return ll1,grad1,hess1,ll2,k1, grad2,hess2,k2 

In [7]:
shi_result = vuong_tests.monte_carlo_shi(100,setup_shi,gen_data)
print(shi_result)

[1. 0. 0.]
